In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

def compute_distance_to_default():
    """
    Solves the Merton model numerically to estimate the firm's asset value,
    asset volatility, and calculate the distance to default (DD) using the following inputs:
    - Equity value: $3 billion
    - Equity volatility: 50%
    - Face value of debt: $20 billion
    - Time to maturity: 3 years
    - Risk-free rate: 4.5%
    
    Returns:
        V_sol (float): estimated asset value (in billions)
        sigma_V_sol (float): estimated asset volatility
        d2 (float): distance to default
    """
    E = 3.0
    sigma_E = 0.50
    D = 20.0
    T = 3.0
    r = 0.045

    def merton_equations(vars):
        V, sigma_V = vars
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
        eq2 = (V * norm.cdf(d1) * sigma_V) / E - sigma_E
        return [eq1, eq2]

    initial_guess = [20.0, 0.3]
    V_sol, sigma_V_sol = fsolve(merton_equations, initial_guess)
    d2 = (np.log(V_sol / D) + (r - 0.5 * sigma_V_sol**2) * T) / (sigma_V_sol * np.sqrt(T))

    return V_sol, sigma_V_sol, d2

V, sigma_V, DD = compute_distance_to_default()
print(f"Estimated Asset Value (V): ${V:.2f} billion")
print(f"Estimated Asset Volatility (σ_V): {sigma_V:.2%}")
print(f"Distance to Default (DD): {DD:.2f}")


Estimated Asset Value (V): $20.22 billion
Estimated Asset Volatility (σ_V): 8.71%
Distance to Default (DD): 0.89


In [2]:
from scipy.stats import norm

def compute_default_probability(dd):
    """
    Computes the default probability using the Merton model
    given the distance to default (DD).

    Args:
        dd (float): Distance to default (d2 in Merton model)

    Returns:
        float: Default probability
    """
    return norm.cdf(-dd)


dd_value = 0.89
default_prob = compute_default_probability(dd_value)
print(f"Default Probability: {default_prob:.4f}")


Default Probability: 0.1867


In [3]:
from scipy.stats import lognorm
import numpy as np

def expected_recovery_rate(V, sigma_V, D, T, r):
    """
    Computes the expected recovery rate given Merton model inputs.

    Args:
        V (float): Current asset value
        sigma_V (float): Asset volatility
        D (float): Face value of debt
        T (float): Time to maturity
        r (float): Risk-free rate

    Returns:
        float: Expected recovery rate (0 to 1)
    """
    mu = np.log(V) + (r - 0.5 * sigma_V**2) * T
    sigma = sigma_V * np.sqrt(T)
    
    dist = lognorm(s=sigma, scale=np.exp(mu))
    
    numerator = dist.expect(lambda x: x, lb=0, ub=D)
    denominator = dist.cdf(D)
    
    return numerator / (D * denominator)

# Using solved values:
V = 20.22
sigma_V = 0.0871
D = 20.0
T = 3.0
r = 0.045

recovery_rate = expected_recovery_rate(V, sigma_V, D, T, r)
print(f"Expected Recovery Rate: {recovery_rate:.4f}")


Expected Recovery Rate: 0.9229
